<a href="https://colab.research.google.com/github/gitpr4596/Sentiment-Classification-/blob/main/Sentiment_Classification_Sem_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score, classification_report

# Load the tweets from the CSV file
tweets_df = pd.read_csv('train.csv')

# Drop any rows with missing values
tweets_df.dropna(subset=['text', 'sentiment'], inplace=True)

# Preprocessing the tweets
tweets = tweets_df['text'].values.tolist()

# Tokenize the tweets
tokenized_tweets = [tweet.split() if isinstance(tweet, str) else [] for tweet in tweets]

# Filter out empty tokenized tweets
tokenized_tweets = [tweet for tweet in tokenized_tweets if tweet]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_tweets, vector_size=100, window=5, min_count=1, workers=4)

# Define a function to calculate the average vector for a tweet
def average_vector(tweet, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in tweet:
        if word in model.wv:
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])
    if num_words != 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Calculate the average word vectors for each tweet
tweet_vectors = np.array([average_vector(tweet, word2vec_model, 100) for tweet in tokenized_tweets])

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_features = tfidf_vectorizer.fit_transform(tweets).toarray()

# Combine Word2Vec and TF-IDF features
X_word2vec = tweet_vectors
X_tfidf = tfidf_features

# Convert sentiment labels to binary (1 for positive, 0 for negative)
y = np.where(tweets_df['sentiment'] == 'positive', 1, 0)

# Split data into training and testing sets for Word2Vec
X_train_word2vec, X_test_word2vec, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Split data into training and testing sets for TF-IDF
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression models
logreg_word2vec = LogisticRegression(max_iter=1000, solver='liblinear')
logreg_tfidf = LogisticRegression(max_iter=1000, solver='liblinear')

# Train Logistic Regression model using Word2Vec
logreg_word2vec.fit(X_train_word2vec, y_train)

# Calculate training accuracy for Word2Vec model
train_predictions_word2vec = logreg_word2vec.predict(X_train_word2vec)
train_accuracy_word2vec = accuracy_score(y_train, train_predictions_word2vec)
print("Training Accuracy (Word2Vec):", train_accuracy_word2vec)

# Calculate testing accuracy for Word2Vec model
test_predictions_word2vec = logreg_word2vec.predict(X_test_word2vec)
test_accuracy_word2vec = accuracy_score(y_test, test_predictions_word2vec)
print("Testing Accuracy (Word2Vec):", test_accuracy_word2vec)

# Print the classification report for the test set using Word2Vec model
print('\nClassification Report for Test Set (Word2Vec):\n', classification_report(y_test, test_predictions_word2vec))

# Train Logistic Regression model using TF-IDF
logreg_tfidf.fit(X_train_tfidf, y_train)

# Calculate training accuracy for TF-IDF model
train_predictions_tfidf = logreg_tfidf.predict(X_train_tfidf)
train_accuracy_tfidf = accuracy_score(y_train, train_predictions_tfidf)
print("\nTraining Accuracy (TF-IDF):", train_accuracy_tfidf)

# Calculate testing accuracy for TF-IDF model
test_predictions_tfidf = logreg_tfidf.predict(X_test_tfidf)
test_accuracy_tfidf = accuracy_score(y_test, test_predictions_tfidf)
print("Testing Accuracy (TF-IDF):", test_accuracy_tfidf)

# Print the classification report for the test set using TF-IDF model
print('\nClassification Report for Test Set (TF-IDF):\n', classification_report(y_test, test_predictions_tfidf))


Training Accuracy (Word2Vec): 0.7298944687045124
Testing Accuracy (Word2Vec): 0.735080058224163

Classification Report for Test Set (Word2Vec):
               precision    recall  f1-score   support

           0       0.74      0.96      0.83      3808
           1       0.71      0.23      0.35      1688

    accuracy                           0.74      5496
   macro avg       0.73      0.59      0.59      5496
weighted avg       0.73      0.74      0.68      5496


Training Accuracy (TF-IDF): 0.8548034934497817
Testing Accuracy (TF-IDF): 0.8518922852983989

Classification Report for Test Set (TF-IDF):
               precision    recall  f1-score   support

           0       0.86      0.94      0.90      3808
           1       0.84      0.64      0.73      1688

    accuracy                           0.85      5496
   macro avg       0.85      0.79      0.81      5496
weighted avg       0.85      0.85      0.85      5496



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import Word2Vec
import numpy as np

# Load the data
tweets_df = pd.read_csv('train.csv')

# Drop any rows with missing values
tweets_df.dropna(subset=['text', 'sentiment'], inplace=True)

# Preprocessing the tweets
tweets = tweets_df['text'].values.tolist()

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(tweets).toarray()

# Convert sentiment labels to binary (1 for positive, 0 for negative)
y = (tweets_df['sentiment'] == 'positive').astype(int)

# Word2Vec Vectorization
tokenized_tweets = [tweet.split() for tweet in tweets]
word2vec_model = Word2Vec(sentences=tokenized_tweets, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.train(tokenized_tweets, total_examples=len(tokenized_tweets), epochs=10)

def get_word2vec_embeddings(tweets, model, vector_size):
    embeddings = []
    for tweet in tweets:
        words = tweet.split()
        tweet_vec = np.zeros(vector_size)
        count = 0
        for word in words:
            if word in model.wv:
                tweet_vec += model.wv[word]
                count += 1
        if count > 0:
            tweet_vec /= count
        embeddings.append(tweet_vec)
    return np.array(embeddings)

X_word2vec = get_word2vec_embeddings(tweets, word2vec_model, 100)

# Split data into training and testing sets for TF-IDF
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Split data into training and testing sets for Word2Vec
X_train_w2v, X_test_w2v, _, _ = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Train SVM model on TF-IDF features
svm_tfidf = SVC()
svm_tfidf.fit(X_train_tfidf, y_train)

# Evaluate the SVM model on TF-IDF features
y_train_pred_tfidf = svm_tfidf.predict(X_train_tfidf)
y_test_pred_tfidf = svm_tfidf.predict(X_test_tfidf)
train_accuracy_tfidf = accuracy_score(y_train, y_train_pred_tfidf)
test_accuracy_tfidf = accuracy_score(y_test, y_test_pred_tfidf)

print("TF-IDF SVM Classification Report:")
print(f"Training Accuracy: {train_accuracy_tfidf}")
print(f"Test Accuracy: {test_accuracy_tfidf}")
print(classification_report(y_test, y_test_pred_tfidf))

# Train SVM model on Word2Vec features
svm_w2v = SVC()
svm_w2v.fit(X_train_w2v, y_train)

# Evaluate the SVM model on Word2Vec features
y_train_pred_w2v = svm_w2v.predict(X_train_w2v)
y_test_pred_w2v = svm_w2v.predict(X_test_w2v)
train_accuracy_w2v = accuracy_score(y_train, y_train_pred_w2v)
test_accuracy_w2v = accuracy_score(y_test, y_test_pred_w2v)

print("\nWord2Vec SVM Classification Report:")
print(f"Training Accuracy: {train_accuracy_w2v}")
print(f"Test Accuracy: {test_accuracy_w2v}")
print(classification_report(y_test, y_test_pred_w2v))


TF-IDF SVM Classification Report:
Training Accuracy: 0.9258551673944687
Test Accuracy: 0.8538937409024745
              precision    recall  f1-score   support

           0       0.86      0.94      0.90      3808
           1       0.84      0.65      0.73      1688

    accuracy                           0.85      5496
   macro avg       0.85      0.80      0.82      5496
weighted avg       0.85      0.85      0.85      5496


Word2Vec SVM Classification Report:
Training Accuracy: 0.7766102620087336
Test Accuracy: 0.7741994177583698
              precision    recall  f1-score   support

           0       0.77      0.96      0.86      3808
           1       0.81      0.34      0.48      1688

    accuracy                           0.77      5496
   macro avg       0.79      0.65      0.67      5496
weighted avg       0.78      0.77      0.74      5496



In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from gensim.models import Word2Vec
from sklearn.metrics import classification_report, confusion_matrix

# Load the data
tweets_df = pd.read_csv('train.csv').dropna(subset=['text', 'sentiment'])

# Preprocessing the tweets
tweets = tweets_df['text'].values.tolist()

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_features = tfidf_vectorizer.fit_transform(tweets).toarray()

# Word2Vec Model
word2vec_model = Word2Vec(sentences=[tweet.split() for tweet in tweets], vector_size=100, window=5, min_count=1, workers=4)

# Function to generate vector representation of a tweet using Word2Vec
def generate_word2vec_vector(tweet):
    vector = np.zeros(word2vec_model.vector_size)
    count = 0
    for word in tweet.split():
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    if count != 0:
        vector /= count
    return vector

# Generate Word2Vec vectors for all tweets
X_word2vec = np.array([generate_word2vec_vector(tweet) for tweet in tweets])

# Convert sentiment labels to binary (1 for positive, 0 for negative)
y = (tweets_df['sentiment'] == 'positive').astype(int)

# Split data into training and testing sets for TF-IDF
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(tfidf_features, y, test_size=0.2, random_state=42)

# Define and train SimpleRNN model with TF-IDF
rnn_model_tfidf = Sequential([
    Embedding(input_dim=1000, output_dim=100, input_length=X_train_tfidf.shape[1]),
    SimpleRNN(100),
    Dense(1, activation='sigmoid')
])
rnn_model_tfidf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model_tfidf.fit(X_train_tfidf, y_train, epochs=50, batch_size=64, validation_data=(X_test_tfidf, y_test))

# Evaluate SimpleRNN model with TF-IDF
train_loss_tfidf, train_accuracy_tfidf = rnn_model_tfidf.evaluate(X_train_tfidf, y_train, verbose=0)
print("Training Accuracy (TF-IDF):", train_accuracy_tfidf)

test_loss_tfidf, test_accuracy_tfidf = rnn_model_tfidf.evaluate(X_test_tfidf, y_test, verbose=0)
print("Testing Accuracy (TF-IDF):", test_accuracy_tfidf)

# Predictions for TF-IDF model
y_pred_tfidf = (rnn_model_tfidf.predict(X_test_tfidf) > 0.5).astype(int)

# Classification report for TF-IDF model
print("Classification Report (TF-IDF):")
print(classification_report(y_test, y_pred_tfidf))

# Split data into training and testing sets for Word2Vec
X_train_word2vec, X_test_word2vec, _, _ = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Reshape the input data for SimpleRNN
X_train_word2vec_rnn = X_train_word2vec.reshape(X_train_word2vec.shape[0], 1, X_train_word2vec.shape[1])
X_test_word2vec_rnn = X_test_word2vec.reshape(X_test_word2vec.shape[0], 1, X_test_word2vec.shape[1])

# Define and train SimpleRNN model with Word2Vec
rnn_model_word2vec = Sequential([
    SimpleRNN(100, input_shape=(1, X_train_word2vec.shape[1])),
    Dense(1, activation='sigmoid')
])
rnn_model_word2vec.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model_word2vec.fit(X_train_word2vec_rnn, y_train, epochs=50, batch_size=64, validation_data=(X_test_word2vec_rnn, y_test))

# Evaluate SimpleRNN model with Word2Vec
train_loss_word2vec, train_accuracy_word2vec = rnn_model_word2vec.evaluate(X_train_word2vec_rnn, y_train, verbose=0)
print("Training Accuracy (Word2Vec):", train_accuracy_word2vec)

test_loss_word2vec, test_accuracy_word2vec = rnn_model_word2vec.evaluate(X_test_word2vec_rnn, y_test, verbose=0)
print("Testing Accuracy (Word2Vec):", test_accuracy_word2vec)

# Predictions for Word2Vec model
y_pred_word2vec = (rnn_model_word2vec.predict(X_test_word2vec_rnn) > 0.5).astype(int)

# Classification report for Word2Vec model
print("Classification Report (Word2Vec):")
print(classification_report(y_test, y_pred_word2vec))


Epoch 1/50
 86/344 [======>.......................] - ETA: 4:03 - loss: 0.6269 - accuracy: 0.6904

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from gensim.models import Word2Vec
from sklearn.metrics import classification_report, confusion_matrix

# Load the data
tweets_df = pd.read_csv('train.csv').dropna(subset=['text', 'sentiment'])

# Preprocessing the tweets
tweets = tweets_df['text'].values.tolist()

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_features = tfidf_vectorizer.fit_transform(tweets).toarray()

# Word2Vec Model
word2vec_model = Word2Vec(sentences=[tweet.split() for tweet in tweets], vector_size=100, window=5, min_count=1, workers=4)

# Generate Word2Vec vectors for all tweets
X_word2vec = np.array([np.mean([word2vec_model.wv[word] for word in tweet.split() if word in word2vec_model.wv], axis=0) for tweet in tweets])

# Convert sentiment labels to binary (1 for positive, 0 for negative)
y = (tweets_df['sentiment'] == 'positive').astype(int)

# Split data into training and testing sets for TF-IDF
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(tfidf_features, y, test_size=0.2, random_state=42)

# Define and train LSTM model with TF-IDF
lstm_model_tfidf = Sequential([
    Embedding(input_dim=1000, output_dim=100, input_length=X_train_tfidf.shape[1]),
    LSTM(100),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])
lstm_model_tfidf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model_tfidf.fit(X_train_tfidf, y_train, epochs=50, batch_size=64, validation_data=(X_test_tfidf, y_test))

# Evaluate LSTM model with TF-IDF
train_loss_tfidf, train_accuracy_tfidf = lstm_model_tfidf.evaluate(X_train_tfidf, y_train, verbose=0)
test_loss_tfidf, test_accuracy_tfidf = lstm_model_tfidf.evaluate(X_test_tfidf, y_test, verbose=0)

print("Training Accuracy (TF-IDF):", train_accuracy_tfidf)
print("Test Accuracy (TF-IDF):", test_accuracy_tfidf)

# Predictions for TF-IDF model
y_pred_tfidf = (lstm_model_tfidf.predict(X_test_tfidf) > 0.5).astype(int)

# Classification report for TF-IDF model
print("Classification Report (TF-IDF):")
print(classification_report(y_test, y_pred_tfidf))

# Split data into training and testing sets for Word2Vec
X_train_word2vec, X_test_word2vec, _, _ = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Reshape the input data for LSTM
X_train_word2vec_lstm = X_train_word2vec.reshape(X_train_word2vec.shape[0], 1, X_train_word2vec.shape[1])
X_test_word2vec_lstm = X_test_word2vec.reshape(X_test_word2vec.shape[0], 1, X_test_word2vec.shape[1])

# Define and train LSTM model with Word2Vec
lstm_model_word2vec = Sequential([
    LSTM(100, input_shape=(1, X_train_word2vec.shape[1])),
    Dense(1, activation='sigmoid')
])
lstm_model_word2vec.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model_word2vec.fit(X_train_word2vec_lstm, y_train, epochs=50, batch_size=64, validation_data=(X_test_word2vec_lstm, y_test))

# Evaluate LSTM model with Word2Vec
train_loss_word2vec, train_accuracy_word2vec = lstm_model_word2vec.evaluate(X_train_word2vec_lstm, y_train, verbose=0)
test_loss_word2vec, test_accuracy_word2vec = lstm_model_word2vec.evaluate(X_test_word2vec_lstm, y_test, verbose=0)

print("Training Accuracy (Word2Vec):", train_accuracy_word2vec)
print("Test Accuracy (Word2Vec):", test_accuracy_word2vec)

# Predictions for Word2Vec model
y_pred_word2vec = (lstm_model_word2vec.predict(X_test_word2vec_lstm) > 0.5).astype(int)

# Classification report for Word2Vec model
print("Classification Report (Word2Vec):")
print(classification_report(y_test, y_pred_word2vec))


Epoch 1/50
344/344 [==============================] - 24s 55ms/step - loss: 0.6249 - accuracy: 0.6864 - val_loss: 0.6178 - val_accuracy: 0.6929
Epoch 2/50
344/344 [==============================] - 14s 41ms/step - loss: 0.6224 - accuracy: 0.6864 - val_loss: 0.6184 - val_accuracy: 0.6929
Epoch 3/50
344/344 [==============================] - 15s 42ms/step - loss: 0.6229 - accuracy: 0.6864 - val_loss: 0.6183 - val_accuracy: 0.6929
Epoch 4/50
344/344 [==============================] - 14s 41ms/step - loss: 0.6227 - accuracy: 0.6864 - val_loss: 0.6205 - val_accuracy: 0.6929
Epoch 5/50
344/344 [==============================] - 14s 41ms/step - loss: 0.6225 - accuracy: 0.6864 - val_loss: 0.6182 - val_accuracy: 0.6929
Epoch 6/50
344/344 [==============================] - 16s 45ms/step - loss: 0.6225 - accuracy: 0.6864 - val_loss: 0.6176 - val_accuracy: 0.6929
Epoch 7/50
344/344 [==============================] - 14s 42ms/step - loss: 0.6222 - accuracy: 0.6864 - val_loss: 0.6168 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/50
344/344 [==============================] - 4s 8ms/step - loss: 0.5997 - accuracy: 0.6987 - val_loss: 0.5706 - val_accuracy: 0.7183
Epoch 2/50
344/344 [==============================] - 2s 6ms/step - loss: 0.5771 - accuracy: 0.7136 - val_loss: 0.5660 - val_accuracy: 0.7207
Epoch 3/50
344/344 [==============================] - 2s 5ms/step - loss: 0.5737 - accuracy: 0.7158 - val_loss: 0.5687 - val_accuracy: 0.7265
Epoch 4/50
344/344 [==============================] - 2s 5ms/step - loss: 0.5718 - accuracy: 0.7169 - val_loss: 0.5629 - val_accuracy: 0.7238
Epoch 5/50
344/344 [==============================] - 2s 5ms/step - loss: 0.5679 - accuracy: 0.7188 - val_loss: 0.5776 - val_accuracy: 0.7189
Epoch 6/50
344/344 [==============================] - 2s 4ms/step - loss: 0.5685 - accuracy: 0.7180 - val_loss: 0.5585 - val_accuracy: 0.7285
Epoch 7/50
344/344 [==============================] - 2s 4ms/step - loss: 0.5670 - accuracy: 0.7200 - val_loss: 0.5571 - val_accuracy: 0.7276
Epoch 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from gensim.models import Word2Vec
from sklearn.metrics import classification_report, confusion_matrix

# Load the data
tweets_df = pd.read_csv('train.csv')

# Drop any rows with missing values
tweets_df.dropna(subset=['text', 'sentiment'], inplace=True)

# Preprocessing the tweets
tweets = tweets_df['text'].values.tolist()

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_features = tfidf_vectorizer.fit_transform(tweets).toarray()

# Word2Vec Model
word2vec_model = Word2Vec(sentences=[tweet.split() for tweet in tweets], vector_size=100, window=5, min_count=1, workers=4)

# Function to generate vector representation of a tweet using Word2Vec
def generate_word2vec_vector(tweet):
    vector = np.zeros(word2vec_model.vector_size)
    count = 0
    for word in tweet.split():
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    if count != 0:
        vector /= count
    return vector

# Generate Word2Vec vectors for all tweets
X_word2vec = np.array([generate_word2vec_vector(tweet) for tweet in tweets])

# Convert sentiment labels to binary (1 for positive, 0 for negative)
y = np.where(tweets_df['sentiment'] == 'positive', 1, 0)

# Split data into training and testing sets for Word2Vec
X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)

# Define and train GRU model for Word2Vec
gru_model_word2vec = Sequential()
gru_model_word2vec.add(Dense(128, activation='relu', input_dim=X_word2vec.shape[1]))
gru_model_word2vec.add(Dense(1, activation='sigmoid'))
gru_model_word2vec.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model_word2vec.fit(X_train_word2vec, y_train_word2vec, epochs=50, batch_size=64, validation_data=(X_test_word2vec, y_test_word2vec))

# Evaluate Word2Vec model
_, train_acc_word2vec = gru_model_word2vec.evaluate(X_train_word2vec, y_train_word2vec)
_, test_acc_word2vec = gru_model_word2vec.evaluate(X_test_word2vec, y_test_word2vec)
print("Word2Vec Model - Training Accuracy:", train_acc_word2vec)
print("Word2Vec Model - Testing Accuracy:", test_acc_word2vec)

# Predictions for Word2Vec model
y_pred_word2vec = (gru_model_word2vec.predict(X_test_word2vec) > 0.5).astype(int)

# Classification report for Word2Vec model
print("Classification Report (Word2Vec):")
print(classification_report(y_test_word2vec, y_pred_word2vec))

# Split data into training and testing sets for TF-IDF
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf_features, y, test_size=0.2, random_state=42)

# Define and train GRU model for TF-IDF
gru_model_tfidf = Sequential()
gru_model_tfidf.add(Dense(128, activation='relu', input_dim=tfidf_features.shape[1]))
gru_model_tfidf.add(Dense(1, activation='sigmoid'))
gru_model_tfidf.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model_tfidf.fit(X_train_tfidf, y_train_tfidf, epochs=50, batch_size=64, validation_data=(X_test_tfidf, y_test_tfidf))

# Evaluate TF-IDF model
_, train_acc_tfidf = gru_model_tfidf.evaluate(X_train_tfidf, y_train_tfidf)
_, test_acc_tfidf = gru_model_tfidf.evaluate(X_test_tfidf, y_test_tfidf)
print("TF-IDF Model - Training Accuracy:", train_acc_tfidf)
print("TF-IDF Model - Testing Accuracy:", test_acc_tfidf)

# Predictions for TF-IDF model
y_pred_tfidf = (gru_model_tfidf.predict(X_test_tfidf) > 0.5).astype(int)

# Classification report for TF-IDF model
print("Classification Report (TF-IDF):")
print(classification_report(y_test_tfidf, y_pred_tfidf))


Epoch 1/50
344/344 [==============================] - 3s 6ms/step - loss: 0.5937 - accuracy: 0.7056 - val_loss: 0.5754 - val_accuracy: 0.7243
Epoch 2/50
344/344 [==============================] - 3s 8ms/step - loss: 0.5755 - accuracy: 0.7149 - val_loss: 0.5632 - val_accuracy: 0.7222
Epoch 3/50
344/344 [==============================] - 3s 9ms/step - loss: 0.5727 - accuracy: 0.7174 - val_loss: 0.5612 - val_accuracy: 0.7269
Epoch 4/50
344/344 [==============================] - 3s 9ms/step - loss: 0.5696 - accuracy: 0.7198 - val_loss: 0.5606 - val_accuracy: 0.7271
Epoch 5/50
344/344 [==============================] - 2s 6ms/step - loss: 0.5692 - accuracy: 0.7204 - val_loss: 0.5579 - val_accuracy: 0.7271
Epoch 6/50
344/344 [==============================] - 1s 3ms/step - loss: 0.5692 - accuracy: 0.7199 - val_loss: 0.5563 - val_accuracy: 0.7293
Epoch 7/50
344/344 [==============================] - 1s 4ms/step - loss: 0.5668 - accuracy: 0.7215 - val_loss: 0.5623 - val_accuracy: 0.7324
Epoch 

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Function to classify sentiment using BERT
def classify_sentiment_bert(user_input):
    # Tokenize input text
    inputs = tokenizer(user_input, return_tensors='pt', truncation=True, padding=True)

    # Perform inference
    outputs = model(**inputs)

    # Get predicted sentiment label
    predicted_class = torch.argmax(outputs.logits).item()

    # Map predicted class to sentiment label
    sentiment = 'positive' if predicted_class == 1 else 'negative'

    return sentiment

# Example usage:
user_input = input("Enter a tweet: ")
sentiment = classify_sentiment_bert(user_input)
print("Sentiment:", sentiment)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Enter a tweet: the movie was great
Sentiment: positive


In [ ]:
from transformers import pipeline
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Load pre-trained sentiment analysis model
sentiment_analysis = pipeline("text-generation", model="gpt2")

# Initialize Sentiment Intensity Analyzer from NLTK
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Function to classify sentiment using GPT-2
def classify_sentiment_gpt2(user_input):
    # Generate text completion using GPT-2 model
    response = sentiment_analysis(user_input + "\nSentiment:", max_length=50, num_return_sequences=1)

    # Extract generated text
    generated_text = response[0]['generated_text'].strip()

    # Analyze sentiment of generated text using NLTK's Sentiment Intensity Analyzer
    sentiment_score = sid.polarity_scores(generated_text)

    # Classify sentiment as positive or negative based on compound score
    if sentiment_score['compound'] >= 0:
        sentiment = "Positive"
    else:
        sentiment = "Negative"

    return sentiment

# Example usage:
user_input = input("Enter a tweet: ")
sentiment = classify_sentiment_gpt2(user_input)
print("Sentiment:", sentiment)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Enter a tweet: The Movie was great


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment: Positive
